In [1]:
import numpy as np
import os
import pandas as pd
import pickle

In [2]:
DATA_FILES = ['WISDM.npz']
MODE = 'LOTO'
BASE_DIR = './data/' + MODE + '/'

In [3]:
for DATA_FILE in DATA_FILES:
    data_input_file = os.path.join(BASE_DIR, DATA_FILE)
    tmp = np.load(data_input_file, allow_pickle=True)
    X = tmp['X']
    X = np.squeeze(X, axis = 1)
    y_one_hot = tmp['y']
    folds = tmp['folds']

    NUM_LABELS = y_one_hot.shape[1]


In [4]:
X.shape

(20846, 100, 3)

In [5]:
y_one_hot.shape

(20846, 6)

In [6]:
folds

array([[array([    0,     1,     2, ..., 20843, 20844, 20845]),
        array([   17,    18,    19, ..., 19787, 19788, 19789])],
       [array([    0,     1,     2, ..., 20843, 20844, 20845]),
        array([  251,   252,   253, ..., 18926, 18927, 18928])],
       [array([    0,     1,     2, ..., 20843, 20844, 20845]),
        array([  119,   120,   121, ..., 20141, 20142, 20143])],
       [array([   17,    18,    19, ..., 20843, 20844, 20845]),
        array([    0,     1,     2, ..., 20588, 20589, 20590])],
       [array([    0,     1,     2, ..., 20843, 20844, 20845]),
        array([  144,   145,   146, ..., 19543, 19544, 19545])],
       [array([    0,     1,     2, ..., 20843, 20844, 20845]),
        array([  131,   132,   133, ..., 17285, 17286, 17287])],
       [array([    0,     1,     2, ..., 20843, 20844, 20845]),
        array([   86,    87,    88, ..., 20465, 20466, 20467])],
       [array([    0,     1,     2, ..., 20843, 20844, 20845]),
        array([  158,   159,   16

In [7]:
folds[0][0].shape

(18393,)

In [8]:
folds[0][1].shape

(2453,)

In [9]:
folds[0][0].shape[0]+folds[0][1].shape[0]

20846

In [10]:
for i in range(len(folds)):
    print(folds[i][0].shape[0]+folds[i][1].shape[0])

20846
20846
20846
20846
20846
20846
20846
20846
20846
20846


In [83]:
data_path = "/media/kavin/silo/ghost-storage/Research/WASH/complex_activities/"

#data_file = "ubicomp08_routines_lda_5_topic_5_doc_nan_no_overlap_doc_topic_chunk_2_unlabeled_filtered_labels.csv"

data_file = "TM_Files/LDA_after_filter/ubicomp08_routines_lda_doc_topic_nan_5-topic_5-doc_5-overlap_2-chunk_1000-num_passes_1-update_every_lda_after_filter_unlabeled.csv"

data_df = pd.read_csv(data_path + data_file)

In [72]:
test_days_list = [3, 1, 7, 2, 6, 5, 4]

In [84]:
ubi_folds = []
for fold_idx in range(len(test_days_list)):
    test_idx = np.where(data_df.day == test_days_list[fold_idx])
    train_idx = np.where(data_df.day != test_days_list[fold_idx])
    ubi_folds.append([train_idx[0],test_idx[0]])
ubi_folds = np.array(ubi_folds)
ubi_folds.shape

(7, 2)

In [85]:
for i in range(len(ubi_folds)):
    print(ubi_folds[i][0].shape[0]+ubi_folds[i][1].shape[0])

514525
514525
514525
514525
514525
514525
514525


In [86]:
test_idx

(array([224257, 224258, 224259, ..., 291933, 291934, 291935]),)

In [87]:
data_df = data_df.fillna(-1)

In [88]:
data_df.topic.unique()

array([ 3.,  2.,  1.,  4., -1.,  0.])

In [89]:
X = np.resize(np.expand_dims(data_df.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]].values, axis = 1), (514525, 1, 4, 3))
y = data_df.iloc[:, 14].values

In [90]:
X.shape

(514525, 1, 4, 3)

In [91]:
X_hours = data_df.iloc[:, [15, 16]].values
X_hours = np.hstack((X_hours,np.zeros((X_hours.shape[0],1))))

In [78]:
X_hours.shape

(514525, 3)

In [65]:
np.expand_dims(X_hours, axis = (1, 2)).shape

(514525, 1, 1, 3)

In [92]:
X_topics = data_df.iloc[:, [19]].values
X_topics = np.hstack((X_topics,np.zeros((X_topics.shape[0],2))))

In [93]:
X_New = np.concatenate((X, np.expand_dims(X_hours, axis = (1, 2))), axis = 2)

In [94]:
X_New = np.concatenate((X_New, np.expand_dims(X_topics, axis = (1, 2))), axis = 2)

In [95]:
X_New.shape

(514525, 1, 6, 3)

In [97]:
y_one_hot = one_hot(y-1, 4)

In [51]:
with open('./data/LOTO/ubicomp_deepconv_attn_baseline_filter_unlabeled_before_LDA.npz', 'wb') as handle:
    pickle.dump({"X": X_New, "y":y_one_hot, "folds":ubi_folds}, handle, protocol=2)


In [70]:
with open('./data/LOTO/ubicomp_deepconv_attn_topic_filter_unlabeled_before_LDA.npz', 'wb') as handle:
    pickle.dump({"X": X_New, "y":y_one_hot, "folds":ubi_folds}, handle, protocol=2)


In [98]:
with open('./data/LOTO/ubicomp_deepconv_attn_topic_filter_unlabeled_before_LDA_separate_topic_channel.npz', 'wb') as handle:
    pickle.dump({"X": X_New, "y":y_one_hot, "folds":ubi_folds}, handle, protocol=2)


In [82]:
with open('./data/LOTO/ubicomp_deepconv_attn_topic_filter_unlabeled.npz', 'wb') as handle:
    pickle.dump({"X": X_New, "y":y_one_hot, "folds":ubi_folds}, handle, protocol=2)


In [19]:
with open('./data/LOTO/ubicomp_deepconv_attn_topic_filter_unlabeled_separate_topic_channel.npz', 'wb') as handle:
    pickle.dump({"X": X_New, "y":y_one_hot, "folds":ubi_folds}, handle, protocol=2)


In [194]:
with open('./data/LOTO/ubicomp_deepconv_attn_topic_all_labels_separate_topic_channel_no_days.npz', 'wb') as handle:
    pickle.dump({"X": X_New, "y":y_one_hot, "folds":ubi_folds}, handle, protocol=2)


In [166]:
with open('./data/LOTO/ubicomp_deepconv_attn_topic_filtered_labels_separate_topic_channel.npz', 'wb') as handle:
    pickle.dump({"X": X_New, "y":y_one_hot, "folds":ubi_folds}, handle, protocol=2)


In [100]:
with open('./data/LOTO/ubicomp_deepconv_attn_baseline_filtered_labels.npz', 'wb') as handle:
    pickle.dump({"X": X_New, "y":y_one_hot, "folds":ubi_folds}, handle, protocol=2)


In [27]:
with open('./data/LOTO/ubicomp_deepconv_attn_topic.npz', 'wb') as handle:
    np.savez(handle, X=X, y=y_one_hot, folds = ubi_folds)

In [1]:
def one_hot(y_, n_values):
    return np.eye(n_values)[np.array(y_, dtype=np.int32)]

In [2]:
from os import walk
import pandas as pd
import numpy as np
import os
import pickle

In [3]:
test_days_list = [3, 1, 7, 2, 6, 5, 4]
data_path = "/Users/kavin/silo/Research/WASH/complex_activities/deepconvlstm_with_attention/diff_windows_no_overlap_2_chunk/"
_, _, filenames = next(walk(data_path))
for data_file in filenames:
    data_df = pd.read_csv(data_path + data_file)
    ubi_folds = []
    for fold_idx in range(len(test_days_list)):
        test_idx = np.where(data_df.day == test_days_list[fold_idx])
        train_idx = np.where(data_df.day != test_days_list[fold_idx])
        ubi_folds.append([train_idx[0],test_idx[0]])
    ubi_folds = np.array(ubi_folds)
    data_len = ubi_folds[1][0].shape[0]+ubi_folds[1][1].shape[0]
    data_df = data_df.fillna(-1)
    print(data_len)
    X = np.resize(np.expand_dims(data_df.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]].values, axis = 1), (data_len, 1, 4, 3))
    y = data_df.iloc[:, 14].values
    X_hours = data_df.iloc[:, [15, 16]].values
    X_hours = np.hstack((X_hours,np.zeros((X_hours.shape[0],1))))
    X_topics = data_df.iloc[:, [19]].values
    X_topics = np.hstack((X_topics,np.zeros((X_topics.shape[0],2))))
    print(X_hours)
    print(X_topics)
    X_New = np.concatenate((X, np.expand_dims(X_hours, axis = (1, 2))), axis = 2)
    X_New = np.concatenate((X_New, np.expand_dims(X_topics, axis = (1, 2))), axis = 2)
    y_one_hot = one_hot(y-1, 4)
    with open('./data/LOTO/{}.npz'.format(data_file.split('.')[0]), 'wb') as handle:
        pickle.dump({"X": X_New, "y":y_one_hot, "folds":ubi_folds}, handle, protocol=2)


773817
[[ 7.  9.  0.]
 [ 7.  9.  0.]
 [ 7.  9.  0.]
 ...
 [ 1. 22.  0.]
 [ 1. 22.  0.]
 [ 1. 22.  0.]]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 ...
 [-1.  0.  0.]
 [-1.  0.  0.]
 [-1.  0.  0.]]
773817
[[ 7.  9.  0.]
 [ 7.  9.  0.]
 [ 7.  9.  0.]
 ...
 [ 1. 22.  0.]
 [ 1. 22.  0.]
 [ 1. 22.  0.]]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 ...
 [-1.  0.  0.]
 [-1.  0.  0.]
 [-1.  0.  0.]]
773817
[[ 7.  9.  0.]
 [ 7.  9.  0.]
 [ 7.  9.  0.]
 ...
 [ 1. 22.  0.]
 [ 1. 22.  0.]
 [ 1. 22.  0.]]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 ...
 [-1.  0.  0.]
 [-1.  0.  0.]
 [-1.  0.  0.]]
773817
[[ 7.  9.  0.]
 [ 7.  9.  0.]
 [ 7.  9.  0.]
 ...
 [ 1. 22.  0.]
 [ 1. 22.  0.]
 [ 1. 22.  0.]]
[[ 2.  0.  0.]
 [ 2.  0.  0.]
 [ 2.  0.  0.]
 ...
 [-1.  0.  0.]
 [-1.  0.  0.]
 [-1.  0.  0.]]
773817
[[ 7.  9.  0.]
 [ 7.  9.  0.]
 [ 7.  9.  0.]
 ...
 [ 1. 22.  0.]
 [ 1. 22.  0.]
 [ 1. 22.  0.]]
[[ 3.  0.  0.]
 [ 3.  0.  0.]
 [ 3.  0.  0.]
 ...
 [-1.  0.  0.]
 [-1.  0.  0.]
 [-1.  0.  0.]]
77381

['ubicomp08_routines_lda_doc_topic_nan_5-topic_3.0-doc_3.0-overlap_2-chunk_1000-num_passes_1-update_every.csv',
 'ubicomp08_routines_lda_doc_topic_nan_5-topic_6.0-doc_6.0-overlap_2-chunk_1000-num_passes_1-update_every.csv',
 'ubicomp08_routines_lda_doc_topic_nan_5-topic_19.0-doc_19.0-overlap_2-chunk_1000-num_passes_1-update_every.csv',
 'ubicomp08_routines_lda_doc_topic_nan_5-topic_29.0-doc_29.0-overlap_2-chunk_1000-num_passes_1-update_every.csv',
 'ubicomp08_routines_lda_doc_topic_nan_5-topic_7.0-doc_7.0-overlap_2-chunk_1000-num_passes_1-update_every.csv',
 'ubicomp08_routines_lda_doc_topic_nan_5-topic_2.0-doc_2.0-overlap_2-chunk_1000-num_passes_1-update_every.csv',
 'ubicomp08_routines_lda_doc_topic_nan_5-topic_18.0-doc_18.0-overlap_2-chunk_1000-num_passes_1-update_every.csv',
 'ubicomp08_routines_lda_doc_topic_nan_5-topic_28.0-doc_28.0-overlap_2-chunk_1000-num_passes_1-update_every.csv',
 'ubicomp08_routines_lda_doc_topic_nan_5-topic_13.0-doc_13.0-overlap_2-chunk_1000-num_passes_1-u